In [8]:
import torch
import torch.nn.functional as F

In [44]:
from torch.utils.cpp_extension import load
!export MAX_JOBS=12
module = load(
    name='m',
    sources=['conv_1d.cu', 'conv_2d.cu', 'main.cpp'],
    verbose=True
)

Using /home/seb/.cache/torch_extensions/py312_cu121 as PyTorch extensions root...
The input conditions for extension module m have changed. Bumping to version 11 and re-building as m_v11...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/seb/.cache/torch_extensions/py312_cu121/m/build.ninja...
/home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module m_v11...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /usr/local/cuda-12.3/bin/nvcc --generate-dependencies-with-compile --dependency-output conv.cuda.o.d -DTORCH_EXTENSION_NAME=m_v11 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/TH -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/THC -isystem /usr/local/cuda-12.3/include -isystem /home/seb/miniconda3/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_61,code=compute_61 -gencode=arch=compute_61,code=sm_61 --compiler-options '-fPIC' -std=c++17 -c /home/seb/CUDA/p

Loading extension module m_v11...


In [67]:
# a = torch.tensor([1.0, 2.0, 3.0, 4.0, 5], dtype=torch.float32, device='cuda')
# mask = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32, device='cuda')
a = torch.rand(512, device='cuda', dtype=torch.float32)
mask_size = 5
mask = torch.rand(mask_size, device='cuda', dtype=torch.float32)


torch_res = F.conv1d(a.unsqueeze(0).unsqueeze(0), mask.unsqueeze(0).unsqueeze(0), padding=mask_size//2).flatten()
my_res = module.conv1d(a, mask).flatten()

print(torch.allclose(my_res, torch_res))
# my_res, torch_res

True
